<a href="https://colab.research.google.com/github/danielbauer1979/FI830/blob/main/HestonAndCoMonteCarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Monte Carlo Pricing in the Heston Model**

This codebook shows how to use Monte Carlo to derive option prices in the Heston model. 

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve

Let's grab the parameters from the previous codebook.

In [ ]:
-

SyntaxError: ignored

Then, let's simulate the combined stock and volatility paths similar to when we simulated the exchange option:

In [ ]:
numbdays = 365
NUM = 20000
Payoff_1 = np.zeros(NUM)
Payoff_2 = np.zeros(NUM)
np.random.seed(101)
stockpaths = np.zeros((NUM,numbdays+1))
volpaths = np.zeros((NUM,numbdays+1))
for i in range(NUM):
  stockpaths[i,0] = S_0
  volpaths[i,0] = v_0
  for t in range (numbdays):
    normal1 = np.random.randn(1)
    normalHelp = np.random.randn(1)
    normal2 = rho * normal1 + np.sqrt(1 - rho * rho) * normalHelp
    stockpaths[i,t+1] = stockpaths[i,t] * (1 + r * T/numbdays + np.sqrt(T/numbdays * np.max([volpaths[i,t],0])) * normal1)
    volpaths[i,t+1] = volpaths[i,t] + kappa * (theta - volpaths[i,t]) * T/numbdays + np.sqrt(T/numbdays * np.max([volpaths[i,t],0])) * volvol * normal2
  Payoff_1[i] = np.max([stockpaths[i,numbdays] - strike,0])
  Payoff_2[i] = np.sqrt(np.max([volpaths[i,numbdays],0]))
Price1 = np.exp(-r*T) * Payoff_1.mean()
print(Price1)
Price2 = np.exp(-r*T) * Payoff_2.mean()
print(Price2)

407.9365599665937
0.16497225172138294


In [ ]:
def BSCall(r_loc,sigma_loc,T_loc,S_loc,K_loc):
  d1 = (np.log(S_loc/K_loc) + (r + sigma_loc*sigma_loc)*T_loc)/(sigma_loc * np.sqrt(T_loc))
  d2 = d1 - sigma_loc * np.sqrt(T_loc)
  return(S_loc * norm.cdf(d1) - K_loc * np.exp(-r * T_loc) * norm.cdf(d2))
BSCall(r,0.1963,T,S_0,strike)

427.0908806842167

In [ ]:
BSP = lambda sig: BSCall(r,sig,T,S_0,strike) - Price1
implied_vol = fsolve(BSP, 0.1963)
print(implied_vol)

[0.18528966]
